<a href="https://colab.research.google.com/github/joydipb01/competitive-coding-agent/blob/main/FineTuning_Unsloth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install unsloth vllm
! pip install --upgrade pillow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.3/47.3 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.3/299.3 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.6/386.6 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.0/169.0 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 76.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 90.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 821.2/821.2 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 78.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 115.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.7/155.7 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.1/

In [2]:
!python --version

Python 3.11.13


In [3]:
from unsloth import FastLanguageModel
import torch

max_sequence_length = 16384
lora_rank = 32

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="NousResearch/Hermes-3-Llama-3.1-8B",
    max_seq_length=max_sequence_length,
    load_in_4bit=True,
    max_lora_rank=lora_rank,
    gpu_memory_utilization=0.5 # This argument is passed to vLLM
)

# The error occurs during the FastLanguageModel.from_pretrained call when it tries to load with vLLM.
# The subsequent call to FastLanguageModel.get_peft_model will not be reached if the first call fails.
# I will keep this part of the code as is, assuming the first part can be fixed by addressing the vLLM issue.
model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank,
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha = lora_rank,
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 07-26 07:28:50 [__init__.py:235] Automatically detected platform cuda.
==((====))==  Unsloth 2025.7.8: Fast Llama patching. Transformers: 4.53.3. vLLM: 0.10.0.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.3.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.31. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/180 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

Unsloth 2025.7.8 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [4]:
import re
from datasets import load_dataset, Dataset
import time

# Load and prep dataset
SYSTEM_PROMPT = """
You will be given a competitive programming problem. Please reason step by step about the solution, then provide a complete implementation in C++17.

Your solution must read input from standard input (cin), write output to standard output (cout).
Do not include any debug prints or additional output.

Respond in the following format:
<think>
...
</think>
```cpp
...
```
"""

XML_COT_FORMAT = """\
<think>
{thinking}
</think>
```cpp
{answer}
```
"""

def extract_xml_answer(text: str) -> str:
    answer = text.split("```cpp")[-1]
    answer = answer.split("```")[0]
    return answer.strip()

def extract_code_answer(text: str) -> str | None:
    if "```" not in text:
        return None
    return text.split("```cpp")[-1].split("```")[0]

# uncomment middle messages for 1-shot prompting
def get_codeforces_questions(split = "train") -> Dataset:
    data = load_dataset('open-r1/codeforces-cots', 'solutions')[split] # type: ignore
    data = data.map(lambda x: { # type: ignore
        'prompt': [
            {'role': 'system', 'content': SYSTEM_PROMPT},
            {'role': 'user', 'content': f"{x['description']}\n{x['input_format']}\n{x['output_format']}\n{x['examples']}"}
        ],
        'answer': extract_code_answer(x['generation'])
    }) # type: ignore
    data = data.remove_columns([col for col in data.column_names if col not in ['prompt', 'answer']])
    return data # type: ignore

dataset = get_codeforces_questions()

# Reward functions
def correctness_reward_func(prompts, completions, answer, **kwargs) -> list[float]:
    responses = [completion[0]['content'] for completion in completions]
    q = prompts[0][-1]['content']
    extracted_responses = [extract_xml_answer(r) for r in responses]
    print('-'*20, f"Question:\n{q}", f"\nAnswer:\n{answer[0]}", f"\nResponse:\n{responses[0]}", f"\nExtracted:\n{extracted_responses[0]}")
    return [2.0 if r == a else 0.0 for r, a in zip(extracted_responses, answer)]

def int_reward_func(completions, **kwargs) -> list[float]:
    responses = [completion[0]['content'] for completion in completions]
    extracted_responses = [extract_xml_answer(r) for r in responses]
    return [0.5 if r.isdigit() else 0.0 for r in extracted_responses]

def strict_format_reward_func(completions, **kwargs) -> list[float]:
    """Reward function that checks if the completion has a specific format."""
    pattern = r"^<think>\n.*?\n</think>\n```cpp\n.*?\n```\n$"
    responses = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, r) for r in responses]
    return [0.5 if match else 0.0 for match in matches]

def count_xml(text) -> float:
    count = 0.0
    if text.count("<think>\n") == 1:
        count += 0.125
    if text.count("\n</think>\n") == 1:
        count += 0.125
    if text.count("\n```cpp\n") == 1:
        count += 0.125
        count -= len(text.split("\n```cpp\n")[-1])*0.001
    if text.count("\n```") == 1:
        count += 0.125
        count -= (len(text.split("\n```")[-1]) - 1)*0.001
    return count

def xmlcount_reward_func(completions, **kwargs) -> list[float]:
    contents = [completion[0]["content"] for completion in completions]
    return [count_xml(c) for c in contents]

README.md: 0.00B [00:00, ?B/s]

solutions/train-00000-of-00010.parquet:   0%|          | 0.00/174M [00:00<?, ?B/s]

solutions/train-00001-of-00010.parquet:   0%|          | 0.00/192M [00:00<?, ?B/s]

solutions/train-00002-of-00010.parquet:   0%|          | 0.00/183M [00:00<?, ?B/s]

solutions/train-00003-of-00010.parquet:   0%|          | 0.00/183M [00:00<?, ?B/s]

...

solutions/train-00007-of-00010.parquet:   0%|          | 0.00/195M [00:00<?, ?B/s]

solutions/train-00008-of-00010.parquet:   0%|          | 0.00/188M [00:00<?, ?B/s]

solutions/train-00009-of-00010.parquet:   0%|          | 0.00/194M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/47780 [00:00<?, ? examples/s]

Map:   0%|          | 0/47780 [00:00<?, ? examples/s]

In [5]:
max_prompt_length = 1024

from trl import GRPOConfig, GRPOTrainer
training_args = GRPOConfig(
    learning_rate = 5e-6,
    adam_beta1 = 0.9,
    adam_beta2 = 0.99,
    weight_decay = 0.1,
    warmup_ratio = 0.1,
    lr_scheduler_type = "cosine",
    optim = "paged_adamw_8bit",
    logging_steps = 1,
    per_device_train_batch_size = 5,
    gradient_accumulation_steps = 4,
    num_generations = 5,
    max_prompt_length = max_prompt_length,
    max_completion_length = max_sequence_length - max_prompt_length,
    # num_train_epochs = 1, # Set to 1 for a full training run
    max_steps = 40,
    save_steps = 40,
    max_grad_norm = 0.1,
    report_to = "none", # Can use Weights & Biases
    output_dir = "outputs",
)

In [6]:
dataset

Dataset({
    features: ['prompt', 'answer'],
    num_rows: 47780
})

In [ ]:
trainer = GRPOTrainer(
    model = model,
    processing_class = tokenizer,
    reward_funcs = [
        xmlcount_reward_func,
        strict_format_reward_func,
        int_reward_func,
        correctness_reward_func,
    ],
    args = training_args,
    train_dataset = dataset,
)
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 47,780 | Num Epochs = 1 | Total steps = 40
O^O/ \_/ \    Batch size per device = 5 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (5 x 4 x 1) = 20
 "-____-"     Trainable parameters = 83,886,080 of 8,114,147,328 (1.03% trained)


-------------------- Question:
Doremy lives in a country consisting of $$$n$$$ cities numbered from $$$1$$$ to $$$n$$$, with $$$a_i$$$ people living in the $$$i$$$-th city. It can be modeled as an undirected graph with $$$n$$$ nodes.

Initially, there are no edges in the graph. Now Doremy wants to make the graph connected.

To do this, she can add an edge between $$$i$$$ and $$$j$$$ if

$$$$$$ \sum_{k \in S} a_k \ge i\cdot j \cdot c, $$$$$$

where $$$S$$$ is the set of all the nodes that are currently in the same connected component of either $$$i$$$ or $$$j$$$, and $$$c$$$ is a given constant.

Can Doremy make the graph connected?

Two nodes $$$(i, j)$$$ are in the same connected component if there exists a path from $$$i$$$ to $$$j$$$. A graph is connected if all its nodes are in the same connected component.
The input consists of multiple test cases. The first line contains a single integer $$$t$$$ ($$$1\le t\le 10^4$$$) — the number of test cases. The description of the test cases 

Step,Training Loss,reward,reward_std,completions / mean_length,completions / min_length,completions / max_length,completions / clipped_ratio,completions / mean_terminated_length,completions / min_terminated_length,completions / max_terminated_length,kl,rewards / xmlcount_reward_func / mean,rewards / xmlcount_reward_func / std,rewards / strict_format_reward_func / mean,rewards / strict_format_reward_func / std,rewards / int_reward_func / mean,rewards / int_reward_func / std,rewards / correctness_reward_func / mean,rewards / correctness_reward_func / std
1,-0.000000,-0.606700,0.345137,447.600006,124.000000,1066.000000,0.000000,447.600006,124.000000,1066.000000,0.000000,-0.606700,0.705599,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


-------------------- Question:
Есть n-подъездный дом, в каждом подъезде по m этажей, и на каждом этаже каждого подъезда ровно k квартир. Таким образом, в доме всего n·m·k квартир. Они пронумерованы естественным образом от 1 до n·m·k, то есть первая квартира на первом этаже в первом подъезде имеет номер 1, первая квартира на втором этаже первого подъезда имеет номер k + 1 и так далее. Особенность этого дома состоит в том, что он круглый. То есть если обходить его по часовой стрелке, то после подъезда номер 1 следует подъезд номер 2, затем подъезд номер 3 и так далее до подъезда номер n. После подъезда номер n снова идёт подъезд номер 1.

Эдвард живёт в квартире номер a, а Наташа — в квартире номер b. Переход на 1 этаж вверх или вниз по лестнице занимает 5 секунд, переход от двери подъезда к двери соседнего подъезда — 15 секунд, а переход в пределах одного этажа одного подъезда происходит мгновенно. Также в каждом подъезде дома есть лифт. Он устроен следующим образом: он всегда приезжает

In [ ]:
text = tokenizer.apply_chat_template([
    {"role" : "user", "content" : "Calculate pi."},
], tokenize = False, add_generation_prompt = True)

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 0.8,
    top_p = 0.95,
    max_tokens = 1024,
)
output = model.fast_generate(
    [text],
    sampling_params = sampling_params,
    lora_request = None,
)[0].outputs[0].text

output